In [41]:
import numpy as np
import pandas as pd
from pathlib import Path

Read both data: meteo and red days

In [42]:
DATA_FOLDER = Path('../data')

df_meteo = pd.read_feather(DATA_FOLDER / f'rtt-vent/10_most_populated_cities_data_rtt_vent_from_20170101.feather')
print('Shape:', df_meteo.shape)
df_meteo.head()

Shape: (48556, 15)


,NOM_USUEL,LON,LAT,AAAAMMJJ,TN,TX,TM,TNTXM,TAMPLI,DG,DRR,FFM,FF2M,FXY,RR
2778237,NICE,7.209,43.648833,20170101,4.6,14.9,8.2,9.8,10.3,0.0,0.0,5.6,NaN,7.7,0.0
2778238,NICE,7.209,43.648833,20170102,5.6,16.1,10.0,10.9,10.5,0.0,0.0,7.0,NaN,11.7,0.0
2778239,NICE,7.209,43.648833,20170103,5.1,14.8,8.9,10.0,9.7,0.0,0.0,5.1,NaN,8.1,0.0
2778240,NICE,7.209,43.648833,20170104,5.4,13.2,7.8,9.3,7.8,0.0,0.0,4.0,NaN,6.8,0.0
2778241,NICE,7.209,43.648833,20170105,4.4,13.0,8.3,8.7,8.6,0.0,0.0,5.0,NaN,9.1,0.0


In [43]:
print('Number of row with a nan value:', df_meteo.isna().sum().sum())

Number of row with a nan value: 56427


In [44]:
# Print nan rows:
print('Nan rows:')
df_meteo[df_meteo.isna().any(axis=1)]

Nan rows:


,NOM_USUEL,LON,LAT,AAAAMMJJ,TN,TX,TM,TNTXM,TAMPLI,DG,DRR,FFM,FF2M,FXY,RR
2778237,NICE,7.209000,43.648833,20170101,4.6,14.9,8.2,9.8,10.3,0.0,0.0,5.6,NaN,7.7,0.0
2778238,NICE,7.209000,43.648833,20170102,5.6,16.1,10.0,10.9,10.5,0.0,0.0,7.0,NaN,11.7,0.0
2778239,NICE,7.209000,43.648833,20170103,5.1,14.8,8.9,10.0,9.7,0.0,0.0,5.1,NaN,8.1,0.0
2778240,NICE,7.209000,43.648833,20170104,5.4,13.2,7.8,9.3,7.8,0.0,0.0,4.0,NaN,6.8,0.0
2778241,NICE,7.209000,43.648833,20170105,4.4,13.0,8.3,8.7,8.6,0.0,0.0,5.0,NaN,9.1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11849170,PARIS-MONTSOURIS,2.337833,48.821667,20241229,0.6,5.0,3.4,2.8,4.4,0.0,0.0,1.5,NaN,2.5,0.0
11849171,PARIS-MONTSOURIS,2.337833,48.821667,20241230,1.4,4.2,2.9,2.8,2.8,0.0,1.0,2.1,NaN,3.6,0.0
11849172,PARIS-MONTSOURIS,2.337833,48.821667,20241231,1.1,5.2,3.7,3.2,4.1,0.0,14.0,3.3,NaN,7.0,0.0
11849173,PARIS-MONTSOURIS,2.337833,48.821667,20250101,4.0,8.8,6.3,6.4,4.8,0.0,327.0,5.5,NaN,9.0,12.9


In [45]:
df_meteo.drop(['DRR', 'DG', 'FF2M', 'FXY'], axis=1, inplace=True)

In [46]:
# Remove duplicated rows:
df_meteo.drop_duplicates(inplace=True)
print('Number of row with a nan value:', df_meteo.isna().sum().sum())
# Print nan rows:
print('Nan rows:')
df_meteo[df_meteo.isna().any(axis=1)]

Number of row with a nan value: 48
Nan rows:


,NOM_USUEL,LON,LAT,AAAAMMJJ,TN,TX,TM,TNTXM,TAMPLI,FFM,RR
2778658,NICE,7.209000,43.648833,20180226,1.0,5.3,NaN,3.2,4.3,3.8,2.0
3599552,MARSEILLE,5.479167,43.310667,20190523,9.0,22.0,NaN,15.5,13.0,1.6,0.0
3599553,MARSEILLE,5.479167,43.310667,20190524,10.0,25.0,NaN,17.5,15.0,NaN,0.0
3599554,MARSEILLE,5.479167,43.310667,20190525,15.0,22.0,NaN,18.5,7.0,NaN,0.2
3599555,MARSEILLE,5.479167,43.310667,20190526,15.0,23.0,NaN,19.0,8.0,NaN,2.0
3599556,MARSEILLE,5.479167,43.310667,20190527,14.0,24.4,NaN,19.2,10.4,NaN,0.0
3600036,MARSEILLE,5.479167,43.310667,20200918,19.9,31.6,25.1,25.8,11.7,NaN,0.0
3600481,MARSEILLE,5.479167,43.310667,20211207,2.7,11.7,7.6,7.2,9.0,NaN,7.2
3600482,MARSEILLE,5.479167,43.310667,20211208,5.7,10.7,7.5,8.2,5.0,NaN,0.2
3600483,MARSEILLE,5.479167,43.310667,20211209,3.0,9.5,5.0,6.3,6.5,NaN,4.4


In [47]:
df_ejp = pd.read_csv(DATA_FOLDER / 'ejp_days-2016-2024.csv', sep=',')
print('Shape:', df_ejp.shape)
df_ejp.tail()

Shape: (169, 2)


,Year,Red Day Date
164,2023-2024,04/03/2024
165,2023-2024,05/03/2024
166,2023-2024,06/03/2024
167,2023-2024,07/03/2024
168,2023-2024,11/03/2024


In [48]:
df_ejp.shape

(169, 2)

Process df_ejp so that the date has the same format as in df_meteo

In [49]:
df_meteo['AAAAMMJJ']

2778237     20170101
2778238     20170102
2778239     20170103
2778240     20170104
2778241     20170105
              ...   
11849170    20241229
11849171    20241230
11849172    20241231
11849173    20250101
11849174    20250102
Name: AAAAMMJJ, Length: 28483, dtype: int64

In [50]:
df_ejp['formatted_red_day_date'] = pd.to_datetime(df_ejp['Red Day Date'], format='%d/%m/%Y').dt.strftime('%Y%m%d')
df_ejp['formatted_red_day_date'] = df_ejp['formatted_red_day_date'].astype(int)
df_ejp['formatted_red_day_date']

0      20161220
1      20170103
2      20170104
3      20170105
4      20170106
         ...   
164    20240304
165    20240305
166    20240306
167    20240307
168    20240311
Name: formatted_red_day_date, Length: 169, dtype: int32

In [51]:
# remove 2016 value
df_ejp = df_ejp[df_ejp['formatted_red_day_date'] >= 20170101]
df_ejp.tail()

,Year,Red Day Date,formatted_red_day_date
164,2023-2024,04/03/2024,20240304
165,2023-2024,05/03/2024,20240305
166,2023-2024,06/03/2024,20240306
167,2023-2024,07/03/2024,20240307
168,2023-2024,11/03/2024,20240311


In [52]:
# Create a new dataframe with the date from 20170101 to 20231231 and whheter or not it was a red day
date_range = pd.date_range(start='2017-01-01', end='2024-03-11')
df_red_days = pd.DataFrame({'AAAAMMJJ': date_range.strftime('%Y%m%d').astype(int)})

df_red_days['is_red_day'] = df_red_days['AAAAMMJJ'].isin(df_ejp['formatted_red_day_date'])

print('Shape:', df_red_days.shape)
df_red_days.head()

Shape: (2627, 2)


,AAAAMMJJ,is_red_day
0,20170101,False
1,20170102,False
2,20170103,True
3,20170104,True
4,20170105,True


In [53]:
df_red_days['is_red_day'].value_counts()

is_red_day
False    2459
True      168
Name: count, dtype: int64

Further process the meteo data so that the format is ready for trainings

In [54]:
# Sort by date and remove data prior to 2017
df_meteo = df_meteo.sort_values('AAAAMMJJ')
df_meteo = df_meteo[df_meteo['AAAAMMJJ'] >= 20170101]

df_meteo = df_meteo.drop_duplicates(keep='first')

df_meteo.head()

,NOM_USUEL,LON,LAT,AAAAMMJJ,TN,TX,TM,TNTXM,TAMPLI,FFM,RR
2778237,NICE,7.209000,43.648833,20170101,4.6,14.9,8.2,9.8,10.3,5.6,0.0
6185595,TOULOUSE-BLAGNAC,1.378833,43.621000,20170101,-2.1,2.7,1.3,0.3,4.8,1.0,0.0
6495310,BORDEAUX-MERIGNAC,-0.691333,44.830667,20170101,2.6,5.0,4.0,3.8,2.4,2.3,0.0
6650371,MONTPELLIER-AEROPORT,3.964667,43.576167,20170101,2.4,12.3,6.7,7.4,9.9,2.1,0.0
7836452,NANTES-BOUGUENAIS,-1.608833,47.150000,20170101,-4.0,5.2,1.0,0.6,9.2,2.9,1.6


In [55]:
# Process NOM_USUEL col to remove values after '-' or '_':
df_meteo['NOM_USUEL'] = df_meteo['NOM_USUEL'].str.split('_|-').str[0]

# Remove LON and LAT cols
df_meteo.drop(['LON', 'LAT'], axis=1, inplace=True)

df_meteo.head()

,NOM_USUEL,AAAAMMJJ,TN,TX,TM,TNTXM,TAMPLI,FFM,RR
2778237,NICE,20170101,4.6,14.9,8.2,9.8,10.3,5.6,0.0
6185595,TOULOUSE,20170101,-2.1,2.7,1.3,0.3,4.8,1.0,0.0
6495310,BORDEAUX,20170101,2.6,5.0,4.0,3.8,2.4,2.3,0.0
6650371,MONTPELLIER,20170101,2.4,12.3,6.7,7.4,9.9,2.1,0.0
7836452,NANTES,20170101,-4.0,5.2,1.0,0.6,9.2,2.9,1.6


In [56]:
# Pivot so that each row is a day and each column is a measure per station
pivoted_df_meteo = df_meteo.pivot(index="AAAAMMJJ", columns="NOM_USUEL")

# Flatten MultiIndex columns
pivoted_df_meteo.columns = ['{}_{}'.format(col[0], col[1]) for col in pivoted_df_meteo.columns]

pivoted_df_meteo.reset_index(inplace=True)
print('Shape:', pivoted_df_meteo.shape)
pivoted_df_meteo.head(10)

Shape: (2924, 71)


,AAAAMMJJ,TN_BORDEAUX,TN_LILLE,TN_LYON,TN_MARSEILLE,TN_MONTPELLIER,TN_NANTES,TN_NICE,TN_PARIS,TN_STRASBOURG,...,RR_BORDEAUX,RR_LILLE,RR_LYON,RR_MARSEILLE,RR_MONTPELLIER,RR_NANTES,RR_NICE,RR_PARIS,RR_STRASBOURG,RR_TOULOUSE
0,20170101,2.6,-4.3,-1.2,4.5,2.4,-4.0,4.6,-4.3,-2.9,...,0.0,1.4,0.0,0.2,0.0,1.6,0.0,0.2,0.0,0.0
1,20170102,2.2,-1.0,-0.5,6.3,3.3,3.8,5.6,-0.6,-1.9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.6,1.0,0.0
2,20170103,1.3,-2.6,-0.8,-0.1,0.3,0.0,5.1,0.9,-3.7,...,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,20170104,-4.9,2.1,-2.0,1.5,-2.8,-3.2,5.4,0.1,-1.9,...,0.0,2.4,1.0,0.0,0.0,0.0,0.0,0.8,0.0,0.0
4,20170105,-2.8,0.3,0.3,2.4,1.6,0.1,4.4,2.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,20170106,-2.2,-4.6,-4.3,-0.7,-1.1,-3.8,2.9,-1.9,-7.1,...,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0
6,20170107,-5.3,-2.6,-7.3,-1.3,-4.2,-4.6,3.1,-3.8,-10.2,...,0.0,1.2,0.0,0.0,0.0,0.0,0.0,0.2,0.8,0.0
7,20170108,-2.6,1.0,-2.4,-1.2,-2.2,3.3,5.0,1.0,-3.0,...,0.0,0.4,0.0,0.0,0.0,1.0,0.0,0.0,0.2,0.0
8,20170109,3.5,4.1,-0.9,1.0,-1.3,6.3,5.0,5.4,-1.3,...,5.0,6.0,1.4,0.0,0.0,3.0,0.0,4.8,1.0,1.8
9,20170110,7.2,2.2,-0.2,-0.5,-1.0,5.7,6.1,4.7,-0.5,...,0.2,0.8,9.0,0.4,0.8,0.0,1.2,2.4,4.0,8.7


Merge dataframes

In [57]:
df_merged = pd.merge(pivoted_df_meteo, df_red_days, on='AAAAMMJJ', how='left')
df_merged.head()

,AAAAMMJJ,TN_BORDEAUX,TN_LILLE,TN_LYON,TN_MARSEILLE,TN_MONTPELLIER,TN_NANTES,TN_NICE,TN_PARIS,TN_STRASBOURG,...,RR_LILLE,RR_LYON,RR_MARSEILLE,RR_MONTPELLIER,RR_NANTES,RR_NICE,RR_PARIS,RR_STRASBOURG,RR_TOULOUSE,is_red_day
0,20170101,2.6,-4.3,-1.2,4.5,2.4,-4.0,4.6,-4.3,-2.9,...,1.4,0.0,0.2,0.0,1.6,0.0,0.2,0.0,0.0,False
1,20170102,2.2,-1.0,-0.5,6.3,3.3,3.8,5.6,-0.6,-1.9,...,0.0,0.0,0.0,0.0,0.0,0.0,1.6,1.0,0.0,False
2,20170103,1.3,-2.6,-0.8,-0.1,0.3,0.0,5.1,0.9,-3.7,...,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
3,20170104,-4.9,2.1,-2.0,1.5,-2.8,-3.2,5.4,0.1,-1.9,...,2.4,1.0,0.0,0.0,0.0,0.0,0.8,0.0,0.0,True
4,20170105,-2.8,0.3,0.3,2.4,1.6,0.1,4.4,2.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True


In [58]:
# Remove data prior to 2024-04
df_merged = df_merged[df_merged['AAAAMMJJ'] <= 20240311]

In [59]:
# Show number of nans:
print('Number of row with a nan value:', df_merged.isna().sum().sum())

Number of row with a nan value: 3099


In [60]:
# show nan rows:
print('Nan rows:')
df_merged[df_merged.isna().any(axis=1)]

Nan rows:


,AAAAMMJJ,TN_BORDEAUX,TN_LILLE,TN_LYON,TN_MARSEILLE,TN_MONTPELLIER,TN_NANTES,TN_NICE,TN_PARIS,TN_STRASBOURG,...,RR_LILLE,RR_LYON,RR_MARSEILLE,RR_MONTPELLIER,RR_NANTES,RR_NICE,RR_PARIS,RR_STRASBOURG,RR_TOULOUSE,is_red_day
0,20170101,2.6,-4.3,-1.2,4.5,2.4,-4.0,4.6,-4.3,-2.9,...,1.4,0.0,0.2,0.0,1.6,0.0,0.2,0.0,0.0,False
4,20170105,-2.8,0.3,0.3,2.4,1.6,0.1,4.4,2.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
149,20170530,16.2,15.7,19.6,13.4,14.6,12.4,19.1,17.1,20.4,...,0.0,0.0,0.2,0.0,0.2,0.0,0.0,3.6,40.0,False
150,20170531,14.5,10.1,18.2,16.0,18.2,12.2,18.8,14.4,15.9,...,0.0,3.0,0.2,0.0,0.0,0.0,0.0,0.0,2.0,False
151,20170601,14.3,12.7,15.7,14.9,18.0,14.4,18.8,15.6,15.2,...,0.0,0.2,0.0,1.4,0.0,0.0,0.0,0.0,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2622,20240307,5.7,NaN,0.3,0.8,0.5,4.0,7.4,3.8,3.6,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
2623,20240308,8.9,NaN,4.8,7.0,8.7,6.9,6.9,3.6,-0.8,...,NaN,0.4,8.0,13.1,1.4,8.1,0.0,0.0,12.9,False
2624,20240309,7.5,NaN,5.9,7.7,9.9,6.2,8.2,6.4,2.3,...,NaN,15.1,69.4,21.8,8.5,40.3,6.5,0.0,0.0,False
2625,20240310,3.3,NaN,5.7,9.2,6.3,0.9,8.4,7.1,6.4,...,NaN,4.4,3.6,0.0,7.5,23.8,0.0,7.3,0.2,False


In [69]:
# print number of nans by column:
print('Number of nans by column:')
pd.set_option('display.max_rows', None) 
df_merged.isna().sum()

Number of nans by column:


AAAAMMJJ                0
TN_BORDEAUX             0
TN_LILLE               71
TN_LYON                 0
TN_MARSEILLE            0
TN_MONTPELLIER        365
TN_NANTES               0
TN_NICE                 0
TN_PARIS                0
TN_STRASBOURG           0
TN_TOULOUSE             0
TX_BORDEAUX             0
TX_LILLE               71
TX_LYON                 0
TX_MARSEILLE            0
TX_MONTPELLIER        365
TX_NANTES               0
TX_NICE                 0
TX_PARIS                0
TX_STRASBOURG           0
TX_TOULOUSE             0
TM_BORDEAUX             0
TM_LILLE               71
TM_LYON                 0
TM_MARSEILLE            5
TM_MONTPELLIER        365
TM_NANTES               0
TM_NICE                 1
TM_PARIS                0
TM_STRASBOURG           0
TM_TOULOUSE             0
TNTXM_BORDEAUX          0
TNTXM_LILLE            71
TNTXM_LYON              0
TNTXM_MARSEILLE         0
TNTXM_MONTPELLIER     365
TNTXM_NANTES            0
TNTXM_NICE              0
TNTXM_PARIS 

In [70]:
df_merged.to_feather(DATA_FOLDER / 'merged_meteo_red_days_from_20170101.feather')